In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Simple Neural Langauge Model 
In this notebook I learned how to create language model from scratch. I followed UMASS Lec of Mohit Iyyer 365. [Lecture](https://www.youtube.com/watch?v=pZt2TLSl7L4&list=PLWnsVgP6CzadmQX6qevbar3_vDBioWHJL&index=6)
In this note book I followed below steps:

1. Created a sentences 
2. Created vocabulary from it with idx 
3. Build a simple neural Network with help of pytorch 
4. Did predictions like predicting next word (3rd since I am using 3 words sentence) 


In [ ]:
# cross entropy loss : it is used in NLMS as well as other classification tasks 
# what is a loss function 
# intuitively : telss us how bad a model is doing at predicting the training data 
# in NLM how bad is the model at predicting the next word 
# assume we have a training ex 
# "Stuedents opened their"=> "books"
# p("books"| "students open their") we want to maximise this probability in our training data 
# we want to minimize the negative log probabilities 
# L = -log(p(books | students opened their))

# why this called cross entroy loss ? 
# cross entory of two distributoins let say p & q 
# quantifies distance between the distributions 
# H(p, q) = -(summation(p(w)logq(w))) this is we can treat as ground truth distribiton 
# p(w) is 1 for books and 0 for every other type 
# q(w) is predicted conditiional probs 
#



In [ ]:
sentences = ['mohit likes icecream', 'Starks were cool', 'life is fool', 'batman is ironman']


In [ ]:
# we will try to predict next word using NLM 

# step one : Tokenization 

vocab = {} # map from word type to index 
inputs = [] # stores an indexified version of each sentences 


for sent in sentences:
    sent_idxes = []
    sent = sent.split() #tokenize w/ whitespace 
    
    for w in sent: 
        if w not in vocab: 
            vocab[w] = len(vocab) # add new type to the vocab 
            
        sent_idxes.append(vocab[w])
    
    inputs.append(sent_idxes)

In [ ]:
print(vocab)

In [ ]:
print(inputs)

In [ ]:
import torch 

# 1 convert to long tensors 
# 2 define inpts and outpts
prefixes = torch.LongTensor([sent[:-1] for sent in inputs])
#print(prefixes)
labels = torch.LongTensor([sent[-1] for sent in inputs])
print('Prefix',prefixes,'Labels', labels)

In [ ]:
# onto defining the network 
import torch.nn as nn

class NLM(nn.Module):
    # important things 
    # 1. write init functions (initializes all teh paarameters of the network )
    # 2. forward functions (defines the forward propagation computations)
    
    
    def __init__(self, d_embedding, d_hidden, window_size, len_vocab):
        super(NLM, self).__init__() # init the base module class 
        self.d_emb = d_embedding
        self.embeddings = nn.Embedding(len_vocab, d_embedding)
        
        # concatenated embeddings > hidden
        self.W_hid = nn.Linear(d_embedding * window_size, d_hidden)
        
        # hidden > output probability distribution over vocab 
        self.W_out = nn.Linear(d_hidden, len_vocab)
        
        
    def forward(self, input):
        batch_size, window_size = input.size()
        embs = self.embeddings(input)
        #print('Embedding size',embs.size())
        
        # next we want to concatenate the prefix emveddings together
        concat_embs = embs.view(batch_size, window_size * self.d_emb)
        #print('concat_embs', concat_embs.size())
        
        #print(embs[0])
        #print(concat_embs[0])

        # now we project thsi to the hidden space 
        hiddens = self.W_hid(concat_embs)
        
#        print('hidden size:',hiddens.size())
        
        
        # finally project hiddens to vocabulary space 
        out = self.W_out(hiddens)
 #       print(out.size())
        
        return out # return unnormalized probability also known as **logits** 
        
        #probs = nn.functional.softmax(out, dim=1)
        #print(probs)
network = NLM(d_embedding=5, d_hidden=12, window_size=2, len_vocab=len(vocab))
    

In [ ]:
print(network)

In [ ]:
logits = network(prefixes)
print(logits)

In [ ]:
# number of epochs : how many times we are going trhough our network 
num_epochs = 100 
learning_rate = 0.1 
loss_fn = nn.CrossEntropyLoss()# it applies log softmax function and then negative log likelihood 

# we will use vanilla gradient descent  you can expreiment with others like adam 
optimizer = torch.optim.SGD(params = network.parameters(), lr=learning_rate)

# training loop 
for i in range(num_epochs):
    logits = network(prefixes)
    loss = loss_fn(logits, labels)
    
    # now let;s update our params to make the loss smaaller 
   # print(loss)

    #Step1 compute gradient 
    loss.backward()
    # step 2 update params using gradient descent 
    optimizer.step()
    
    # zero the gradients for next epoch
    optimizer.zero_grad()
    print(f'Epoch {i}, loss {loss}')


In [ ]:
# is it working
# reverse vocabulary mapping (idx> word type)

rev_vocab =dict((idx, word) for (word, idx) in vocab.items()) 

In [ ]:
rev_vocab

In [ ]:
mohitlikes = prefixes[0].unsqueeze(0)
logits = network(mohitlikes)
#print(logits)
prob = nn.functional.softmax(logits, dim=1).squeeze()

print(prob)

In [ ]:
argmax_idx = torch.argmax(prob).item()
print('given mohit likes model predicts "%s" as the next word with %0.4f probability '%(rev_vocab[argmax_idx], prob[argmax_idx]))

In [ ]:
lifeis = prefixes[2].unsqueeze(0)
logits = network(lifeis)
#print(logits)
prob = nn.functional.softmax(logits, dim=1).squeeze()

print(prob)
argmax_idx = torch.argmax(prob).item()
print('given "life is" model predicts "%s" as the next word with %0.4f probability '%(rev_vocab[argmax_idx], prob[argmax_idx]))

In [ ]:
# let's no use a dataset to Generate new text with help of GRU


# TASK 2
    let's no use a dataset to Generate new text with help of GRU


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import nltk 
import string 
import unidecode 
import random 
import torch 


In [ ]:
# to check the GPU is working or not 
device = torch.cuda.is_available()
if device : 
    print('GPU is available')
else : 
    print("we are working on CPU ;P")

In [ ]:
# let's load spooky data set

train = pd.read_csv('../input/spooky/train.csv')
train.head()

In [ ]:
train.author.value_counts()

In [ ]:
# we will use MWS only for this task 
text = list(train[train['author']=='MWS']['text'][:2000])

In [ ]:
text[:10]

In [ ]:
def joinString(text):
    return ' '.join(s for s in text)
text = joinString(text)
len(text.split())

In [ ]:
text = text.lower().split()

In [ ]:
trigrams = [([text[i], text[i+1]], text[i+2]) for i in range(len(text) - 2)]

In [ ]:
trigrams[:2]

In [ ]:
vocab = set(text)
print(len(vocab))

In [ ]:
voc_len = len(vocab)
word_to_idx = { word: i for i, word in enumerate(vocab)}

In [ ]:
inp = []
tar = []
for context, target in trigrams : 
    context_idxs = torch.tensor([word_to_idx[w] for w in context], dtype=torch.long)
    inp.append(context_idxs)
    targ = torch.tensor([word_to_idx[target]], dtype=torch.long)
    tar.append(targ)

In [ ]:
# Let's build our model now 
import torch
import torch.nn as nn 
from torch.autograd import Variable 

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size 
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size*2, hidden_size, n_layers, batch_first=True, bidirectional=False)
        self.decoder = nn.Linear(hidden_size, output_size)
        
        
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden
    
    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [ ]:
rnn = RNN(2, 10, 1)

In [ ]:
print(rnn)

In [ ]:
# training 
chunklen = len(trigrams)
def train(inp , target):
    hidden = decoder.init_hidden().cuda()
    decoder.zero_grad()
    loss = 0
    
    for c in range(chunklen):
        output, hidden = decoder(inp[c].cuda(), hidden)
        loss += criterion(output, target[c].cuda())
        
    loss.backward()

    decoder_optimizer.step()
    
    return loss.data.item()/chunklen
        
        

In [ ]:
import time, math 
def time_since(since):
    s = time.time() - since
    m = math.floor(s/60)
    s -= m * 60
    
    return '%dm %ds' %(m, s)

In [ ]:
n_epochs =40
print_every = 5
plot_every = 1
hidden_size = 100
n_layers = 1 
lr = 0.015 

decoder = RNN(voc_len, hidden_size, voc_len, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
start = time.time()
all_losses = []
loss_avg = 0
if(device):
    decoder.cuda()
for epoch in range(1, n_epochs+1): 
    loss = train(inp, tar)
    loss_avg  +=loss
    
    if epoch % print_every ==0:
        print("[%s (%d %d%%) %.4f]" %(time_since(start), epoch, epoch/n_epochs * 50, loss))
    
    if epoch % plot_every == 0: 
        all_losses.append(loss_avg/plot_every)
        loss_avg = 0

In [ ]:
def inference(prime_str='ok so', predict_len=100, temp = 0.8):
    hidden = decoder.init_hidden().cuda()
    
    for p in range(predict_len):
        prime_input = torch.tensor([word_to_idx[w] for w in prime_str.split()], dtype=torch.long).cuda()
        inp = prime_input[-2:]
        output, hidden = decoder(inp, hidden)
        
        output_dist = output.data.view(-1).div(temp).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # add predicted word to string and use as next input
        predicted_word = list(word_to_idx.keys())[list(word_to_idx.values()).index(top_i)]
        prime_str +=" "+predicted_word
        
    return prime_str

In [ ]:
print(inference("i am", 40, temp=1))

In [ ]:
print(inference("proud of", 40))